In [1]:
import os
import importlib
import time

import numpy as np
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import skimage
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
#from torchvision import transforms
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler

import load_data
import model

In [2]:
plt.style.use('Solarize_Light2')
%matplotlib inline
importlib.reload(load_data)
importlib.reload(model)

<module 'model' from '/home/uldo/work/kaggle/competitions/Humpback_Whale_Identification/code/model.py'>

In [3]:
BATCH_SIZE = 40

In [4]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        """ The original code didn't expect gray scale images """
        gray_scale_image = torch.zeros([image_size,image_size]).shape == image.shape
        if gray_scale_image:
            image = np.stack((image,)*3, axis=-1)
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'label': torch.tensor(label)}

In [5]:
def prepare_loaders(dataset, valid_train_ratio=0.6):
    dataset_size = len(dataset)
    print("dataset_size: ", dataset_size)

    validation_subset_size = int(dataset_size * (1 - valid_train_ratio))
    print("validation_subset_size: ", validation_subset_size)

    indices = list(range(dataset_size))
    validation_indices = np.random.choice(indices, size=validation_subset_size, replace=False)
    train_indices = list(set(indices) - set(validation_indices))

    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)
    
    dataset_sizes = {
            'train': len(train_indices),
            'validation': len(validation_indices)
        }

    train_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=train_sampler)
    validation_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=validation_sampler)

    loaders = {
            'train': train_loader,
            'validation': validation_loader
        }

    return loaders, dataset_sizes

In [6]:
train_df = load_data.load_text_data('../input/train.csv')

In [7]:
dataset = load_data.HumpbackWhalesDataset(
    train_df,
    #transform=load_data.transforms.ToTensor()
    transform=ToTensor()
)

In [8]:
dataset_loaders, dataset_sizes = prepare_loaders(dataset)

dataset_size:  25361
validation_subset_size:  10144


In [9]:
train_loader = dataset_loaders['train']
validation_loader = dataset_loaders['validation']
train_dataset_size = dataset_sizes['train']
validation_dataset_size = dataset_sizes['validation']

In [10]:
print(train_dataset_size)
print(validation_dataset_size)

15217
10144


In [11]:
print(train_loader)

In [12]:
train_iter = iter(train_loader)

In [13]:
print(type(train_iter))

<class 'torch.utils.data.dataloader._DataLoaderIter'>


In [14]:
images, labels = train_iter.next()

NameError: Traceback (most recent call last):
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/uldo/work/kaggle/competitions/Humpback_Whale_Identification/code/load_data.py", line 195, in __getitem__
    sample = self.transform(sample)
  File "<ipython-input-4-9ef42ae72fe5>", line 11, in __call__
    gray_scale_image = torch.zeros([image_size,image_size]).shape == image.shape
NameError: name 'image_size' is not defined
